In [ ]:
#|hide
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#| default_exp ipyleaflet

# ipyleaflet module

> Extends ipyleaflet objects to work with Earth Engine generated data.

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import ee
import json
import ipyleaflet
import ipytree
import ipywidgets as widgets
import logging
import traitlets

In [ ]:
#|export
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

## Map

In [ ]:
#|export
DEFAULT_MAP_HEIGHT = '400px'

class TileLayerEE(ipyleaflet.TileLayer):
  """Class for a tile layer generated by Earth Engine.
  
  Attributes:
    ee_object: An Earth Engine object.
  """
  def __init__(self,
               ee_object:ee.Image, # An Earth Engine Image object
               *args,
               **kwargs):

    self.ee_object = ee_object

    super(TileLayerEE, self).__init__(*args, **kwargs)


class Map(ipyleaflet.Map):
  """An interactive map class for Jupyter clients.
  
  Attributes:
    layers_control: a boolean indicating whether to display a layers control.
  """

  layers_control = traitlets.Bool(True)

  def __init__(self, *args, **kwargs):

    self.layers_control_instance = None

    # Set default values for the map.
    if 'zoom' not in kwargs:
      kwargs['zoom'] = 4
    
    if 'basemap' not in kwargs:
      kwargs['basemap'] = ipyleaflet.basemap_to_tiles(ipyleaflet.basemaps.Stamen.Watercolor)

    if 'height' not in kwargs:
      kwargs['height'] = DEFAULT_MAP_HEIGHT
    
    super(Map, self).__init__(*args, **kwargs)
    
    if self.layers_control:
      self.layers_control_instance = ipyleaflet.LayersControl(position='topright')
      self.add_control(self.layers_control_instance)
        
    self.default_style.cursor = 'crosshair'
    
    self.layout.width = '100%'
    
    
  def addLayer(self, eeObject, visParams={}, name=None, shown=True, opacity=1):
    """Adds a layer for an Earth Engine object."""
    
    if name is None:
      # Provide a default name for the layer in the form "Layer ##"
      name = f'Layer {len(self.layers)}'

    def get_tile_layer_url(ee_image_object):
      map_id_dict = ee.Image(ee_image_object).getMapId()
      return map_id_dict['tile_fetcher'].url_format

    # Assume that the eeObject is an ee.Image.
    # TODO: Generalize this to other EE objects.
    ee_image = eeObject

    tile_url = get_tile_layer_url(
      ee_image.visualize(**visParams)
    )
    self.add_layer(TileLayerEE(ee_object=eeObject, url=tile_url, name=name, visible=shown))

Display the default interactive map.

In [ ]:
map = Map()
map

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

Display an Earth Engine raster layer.

In [ ]:
ee.Initialize()

In [ ]:
dem = ee.Image('USGS/SRTMGL1_003')
sample_image = dem.updateMask(dem.gt(0))

map2 = Map()
map2.addLayer(sample_image, {'min':0, 'max':4000}, name="My layer", shown=True)
map2

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Map Inspector

The following defines a `StructureTree` class, which builds a tree structure corresponding to data structure provided as input. The tree branches are lazily populated as the tree is expanded, so that large data structures can be efficiently rendered.

In [ ]:
#|export
class StructureTree(ipytree.Tree):

    JSON_PREFIX = 'JSON: '
    
    def __init__(self, data, **kwargs):
        self.data = data
        super().__init__(StructureTree.__ipytreeify(data))
        
    @staticmethod
    def __ipytreeify(data) -> tuple:
        """Return a sequence of nodes"""

        def is_json(in_str):
            '''Determines if a string is JSON.'''
            return in_str.startswith(StructureTree.JSON_PREFIX)

        def inline_style(x, color='blue'):
            '''Wrap a string with inline HTML styling.'''
            return f'<B><SPAN style="color:{color}">{x}</SPAN></B>'
    
        def handle_node_open(change):
            if change['new']:
                nodes_unpacked = []
                for node in change['owner'].nodes:
                    # If there no subnodes, try to populate the subnodes.
                    if len(node.nodes) == 0:
                        if is_json(node.name):
                          unpacked_json = json.loads(node.name[len(StructureTree.JSON_PREFIX):])
                          if isinstance(unpacked_json, list):
                              nodes_unpacked = StructureTree.__ipytreeify(unpacked_json)
                          elif isinstance(unpacked_json, dict):
                              nodes_unpacked = StructureTree.__ipytreeify(unpacked_json)
                          else:
                              raise
                        else: # ~is_json(node.name)
                          nodes_unpacked.append(node)
                        change['owner'].nodes = nodes_unpacked    
    
        if isinstance(data, list):
            node_list = []
            for count, el in enumerate(data):
                if isinstance(el, list):
                    subnode = ipytree.Node(
                        name=f'{inline_style("List")} ({len(el)} elements)',
                        nodes=[ipytree.Node(f'{StructureTree.JSON_PREFIX}{json.dumps(el)}')],
                        opened=False)
                    subnode.observe(handle_node_open, names='opened')
                elif isinstance(el, dict):
                    subnode = ipytree.Node(
                        name=f'{inline_style("Object")} ({len(el)} elements)',
                        nodes=[ipytree.Node(f'{StructureTree.JSON_PREFIX}{json.dumps(el)}')],
                        opened=False)
                    subnode.observe(handle_node_open, names='opened')   
                else:
                    subnode = ipytree.Node(f'{el}')
                node_list.append(subnode)
            return node_list
        elif isinstance(data, dict):
            node_list = []
            for key, value in data.items():
                if isinstance(value, list):
                    subnode = ipytree.Node(
                        name=f'{inline_style(key)}: List ({len(value)} elements)',
                        nodes=[ipytree.Node(f'{StructureTree.JSON_PREFIX}{json.dumps(value)}')],
                        opened=False)
                    subnode.observe(handle_node_open, names='opened')  
                elif isinstance(value, dict):
                    subnode = ipytree.Node(
                        name=f'{inline_style(key)}: Object ({len(value)} elements)',
                        nodes=[ipytree.Node(f'{StructureTree.JSON_PREFIX}{json.dumps(value)}')],
                        opened=False)
                    subnode.observe(handle_node_open, names='opened')  
                else:
                    subnode = ipytree.Node(f'{inline_style(key)}: {value}')
                node_list.append(subnode)
            return node_list
        else:
            return (data, )


Here is a `StructureTree` object that demonstrates how lists, dictionaries, and nested objects are rendered:

In [ ]:
st = StructureTree({'none': None,
               'number': 1,
               'string': 'hello!',
               'list': [1, 'item1'], 
               'dictionary': {'a key': 'a value'},
               'nesting': [
                   ['nested', 'list'],
                   {'a nested': 'dictionary'}
               ]})
st

StructureTree(nodes=(Node(name='<B><SPAN style="color:blue">none</SPAN></B>: None'), Node(name='<B><SPAN style…

The following defines an `Inspector` class, which is used to display information obtained from querying a `Map` object.

In [ ]:
#|export

# Map scale at Level 0 in meters/pixel
SCALE_LEVEL_0 = 156543.03392

#class Inspector(ipytree.Tree):
class Inspector(StructureTree):   
    """Class representing an inspector tool that responds to map events."""
    
    def __init__(self,
                 map_object=None, # An Earth Engine Image object
                 *args,
                 **kwargs):
        
        self.point_data = None
        # self.tree_data = None
        self.objects_data = {}
        self.pixels_data = {}
        
        self._disabled = False
        
        self.map_object = map_object
        self.layout.width = '100%'
        self.layout.max_height = '400px'
        self.layout.overflow = 'scroll'

        super(Inspector, self).__init__(
            data=['Click on the map to inspect the layers.'],
            *args, 
            **kwargs
        )

        if map_object:
            self.set_map(map_object)
            
        # self.update_inspector()
    
    @property
    def point_node(self):
        return self.nodes[0]
    
    @point_node.setter
    def point_node(self, new_point_node):
        #(lat, lon) = new_coords
        _temp_nodes = list(self.nodes)
        _temp_nodes[0] = new_point_node
        self.nodes = _temp_nodes
    
    @property
    def pixels_node(self):
        return self.nodes[1]
    
    @property
    def objects_node(self):
        return self.nodes[2]
    
    @property
    def disabled(self):
        return self._disabled
    
    @disabled.setter
    def disabled(self, state):
        self._disabled = state
    
    def update_inspector(self, coords=None):
        """Update information in the inspector tree."""
            
        def _order_items(item_dict, ordering_list):
            """Orders dictionary items in a specified order."""
            list_of_tuples = [(key, item_dict[key]) for key in [x for x in ordering_list if x in item_dict.keys()]]
            return dict(list_of_tuples)  
    
        if self.map_object:
            if not self.disabled and coords:
                
                self.nodes = [ipytree.Node('<I>Loading...</I>')]
                
                (lat, lon) = coords

                scale_approx = SCALE_LEVEL_0 / 2**self.map_object.zoom
                self.point_data = {
                    'Longitude': f'{lon:.6f}',
                    'Latitude': f'{lat:.6f}',
                    'Zoom Level': f'{self.map_object.zoom:.0f}',
                    'Scale (approx. m/px)': f'{scale_approx:.2f}'
                }
            
                # Update the Pixels folder
                for layer in self.map_object.layers:
                    if not layer.base:
                        ee_type = ee.Algorithms.ObjectType(layer.ee_object).getInfo()

                        if ee_type == 'Image':
                            value_dict = layer.ee_object.reduceRegion(
                                    reducer=ee.Reducer.mean(),
                                    geometry=ee.Geometry.Point(lon, lat),
                                    scale=scale_approx,
                                    bestEffort=True
                                ).getInfo()
                            num_bands = len(value_dict.keys())
                            
                            band_dict = {}
                            has_unmasked_pixel = False
                            for bandname in layer.ee_object.bandNames().getInfo():          
                                if value_dict[bandname] is not None:
                                    has_unmasked_pixel = True
                                    band_dict.update({f'{bandname}': f'{value_dict[bandname]:.4f}'})
                                else: 
                                    band_dict.update({f'{bandname}': f'<SPAN style="color:grey">masked</SPAN>'})
                            
                            # if not has_unmasked_pixel:
                            #     layer_node.nodes = [
                            #         ipytree.Node(f'No unmasked pixels at clicked point.'),
                            #     ] 
                            # pixel_nodes.append(layer_node)   
                            self.pixels_data.update({f'{layer.name}': band_dict})
            
                # Update the Objects folder
                for layer in self.map_object.layers:
                    if not layer.base:
                        ee_type = ee.Algorithms.ObjectType(layer.ee_object).getInfo()                    
                        layer_info = layer.ee_object.getInfo()

                        # Order the layer information.
                        ordering_list = ['type', 'id', 'version', 'bands', 'properties']
                        layer_info = _order_items(layer_info, ordering_list)
                        layer_dict = {f'{layer.name}': layer_info}
                        
                        self.objects_data.update(layer_dict)
                    
                self.nodes = [
                    ipytree.Node('<B>Point</B>', nodes=StructureTree(self.point_data).nodes),
                    ipytree.Node('<B>Pixels</B>', nodes=StructureTree(self.pixels_data).nodes, opened=False),
                    ipytree.Node('<B>Objects</B>', nodes=StructureTree(self.objects_data).nodes, opened=False),
                ]                
    
    def register_map(self, map_object):
        def handle_interaction(type, event, coordinates):
            if type == 'click':
                self.update_inspector(coordinates)
        map_object.on_interaction(handle_interaction)
            
    def set_map(self, map_object):
        self.map_object = map_object
        self.register_map(map_object)

    def get_map(self):
        return self.map_object

In [ ]:
# Testing it out...
map1 = Map()
map1.addLayer(ee.Image.pixelLonLat(), {'min':-90, 'max':90, 'opacity':0.5}, 'pixelLatLon')
# map1.addLayer(ee.Image("CGIAR/SRTM90_V4"), {'opacity':0.5}, 'SRTM')
inspector1 = Inspector(map_object = map1)

zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=7)
widget_control1 = ipyleaflet.WidgetControl(widget=inspector1,
                                           position='bottomright',
                                           layout=widgets.Layout(overflow='scroll'))
map1.add_control(widget_control1)
map1

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Examples

### Inspector object only

In [ ]:
inspector1 = Inspector()
inspector1

Inspector(layout=Layout(max_height='400px', overflow='scroll', width='100%'), nodes=(Node(name='Click on the m…

### Inspector with a Map

In [ ]:
ee.Initialize()

In [ ]:
map = Map()
inspector2 = Inspector(map_object=map)
display(
    widgets.HBox([
        map,
        inspector2
    ],
    layout=widgets.Layout(border='1px solid black')))

map.addLayer(ee.Image.pixelLonLat(), {'min':-90, 'max':90, 'opacity':0.5}, 'LonLat')
map.addLayer(
    ee.Image('LANDSAT/LC09/C02/T1_L2/LC09_187058_20220105'),
    {'min':0, 'max':90, 'opacity':0.5},
    'Landsat')

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()